# 0. Install Dependencies

# 1. Test Random Environment with OpenAI Gym

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
class BTCTradingEnv(Env):
    def __init__(self, df, lookback=48, starting_portfolio=100, commitions=0.05, columns=['open', 'pOpen', 'pHigh', 'pLow', 'pClose'], lookback_window=50):
        self.action_space = Discrete(3)
        self.observation_space = Box(shape=(1, df[columns].shape[1]),
                                     low=-1.0, high=1.0,
                                     dtype=np.float32)
        self.current_step = 0

        self.lookback_window = lookback_window

        c = [*columns]
        c1 = [*columns]
        c1.remove('open')
        self.columns = c
        self.predict_columns = c1

        self.starting_portfolio = starting_portfolio
        self.portfolio = self.starting_portfolio
        self.balance = self.portfolio
        self.prev_balance = self.balance
        self.commitions = 1-commitions
        self.coins = 0

        self.df = df
        
        self.hours_in_week = 24 * 7
        self.rand_start = random.randrange(0, self.df.shape[0] - self.hours_in_week)
        self.trading_week = df.iloc[self.rand_start:self.rand_start+self.hours_in_week]

        self.bought = False
        self.bought_price = 0
        self.sallen = True
        self.state = None

        self.action = 0

        self.get_state()


    def get_state(self):
        self.state = self.trading_week.iloc[self.current_step][self.predict_columns].values.reshape(-1,1)


    def buy(self, price):
        if not self.bought:
            self.bought_price = price
            self.coins = self.portfolio * self.commitions / self.bought_price
            self.portfolio = 0
            self.bought = True
            self.sallen = False


    def sell(self, price):
        if not self.sallen:
            self.portfolio = self.coins * price * self.commitions
            self.coins = 0
            self.bought = False
            self.sallen = True


    def calculate_balance(self, price):
        self.balance = self.portfolio + self.coins * price


    def step(self, action):
        price = self.trading_week.iloc[self.current_step]['open']

        self.action = action

        if self.action == 0:
            self.buy(price)
        elif self.action == 1:
            self.sell(price)
        elif self.action == 2:
            pass

        self.calculate_balance(price)

        
        reward = self.balance - self.prev_balance
        self.prev_balance = self.balance
        # Reduce shower length by 1 second
        self.current_step += 1
        
        # Check if shower is done
        end_df = self.current_step > self.trading_week.shape[0] - 1
        bankrupt = self.balance <= 10
        done = end_df or bankrupt
        

        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {'end_df' : end_df,
                'bankrupt' : bankrupt,
                'reward': reward}
        
        # Return step information
        return self.state, reward, done, info


    def render(self):
        # Implement viz
        actions = ['buy', 'sell', 'hold']
        print(f'''balance : {self.balance} - diff : {self.balance - self.starting_portfolio} - account : {self.portfolio} - coins : {self.coins} - {actions[self.action]}''')

        # if self.visualization == None:
        #     self.visualization = StockTradingGraph(self.df, "BTC")
            
        # if self.current_step > self.lookback_window:        
        #     self.visualization.render(self.current_step, self.net_worth, 
        #                               self.trades, window_size=self.lookback_window)
    

    def reset(self):
        self.portfolio = self.starting_portfolio
        self.coins = 0

        self.rand_start = random.randrange(0, self.df.shape[0] - self.hours_in_week)
        self.trading_week = self.df.iloc[self.rand_start:self.rand_start+self.hours_in_week]

        self.bought = False
        self.sallen = True

        self.current_step = 0

        self.get_state()
        return self.state
    

In [3]:
env = BTCTradingEnv(pd.read_csv('./data/binance/cleaned/Binance_BTCUSDT_1h.csv', index_col='unix'),
                    commitions=0.005,
                    starting_portfolio=1000,
                    columns=['open', 'pOpen', 'pHigh', 'pLow', 'pClose'])

In [4]:
episodes = 1
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        # print('before')
        # env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        # print('after')
    score = env.balance - env.starting_portfolio
    print('Episode:{} Score:{}'.format(episode, score))
    print(info)

Episode:1 Score:-154.4662436482222
{'end_df': True, 'bankrupt': False, 'reward': 0.0}


# 2. Create a Deep Learning Model with Keras

In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [6]:
states = env.observation_space.shape
actions = env.action_space.n

In [7]:
env.observation_space

Box([[-1. -1. -1. -1. -1.]], [[1. 1. 1. 1. 1.]], (1, 5), float32)

In [17]:
print(f'actions : {actions}')
print(f'states : {states}')

actions : 3
states : (1, 5)


In [30]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

# 3. Build Agent with Keras-RL

In [31]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [32]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [33]:
model = build_model(states, actions)


In [34]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 1, 24)             144       
                                                                 
 dense_7 (Dense)             (None, 1, 24)             600       
                                                                 
 dense_8 (Dense)             (None, 1, 3)              75        
                                                                 
Total params: 819
Trainable params: 819
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.input.shape

TensorShape([None, 1, 5])

In [38]:
model.predict(np.array([0,0,0,0,0], dtype=np.float32).reshape((1,1,5)))

C:\Users\aless\Anaconda3\envs\qlearning\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


array([[[0., 0., 0.]]], dtype=float32)

In [39]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mse'])
dqn.fit(env, nb_steps=50000, visualize=True, verbose=1)

ValueError: Model output "Tensor("dense_8/BiasAdd:0", shape=(None, 1, 3), dtype=float32)" has invalid shape. DQN expects a model that has one dimension for each action, in this case 3.

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -56.000, steps: 60
Episode 2: reward: -60.000, steps: 60
Episode 3: reward: -50.000, steps: 60
Episode 4: reward: -60.000, steps: 60
Episode 5: reward: -56.000, steps: 60
Episode 6: reward: -52.000, steps: 60
Episode 7: reward: -60.000, steps: 60
Episode 8: reward: -50.000, steps: 60
Episode 9: reward: -52.000, steps: 60
Episode 10: reward: -56.000, steps: 60
Episode 11: reward: -60.000, steps: 60
Episode 12: reward: -60.000, steps: 60
Episode 13: reward: -52.000, steps: 60
Episode 14: reward: -52.000, steps: 60
Episode 15: reward: -58.000, steps: 60
Episode 16: reward: -50.000, steps: 60
Episode 17: reward: -54.000, steps: 60
Episode 18: reward: -58.000, steps: 60
Episode 19: reward: -60.000, steps: 60
Episode 20: reward: -56.000, steps: 60
Episode 21: reward: -56.000, steps: 60
Episode 22: reward: -52.000, steps: 60
Episode 23: reward: -60.000, steps: 60
Episode 24: reward: -56.000, steps: 60
Episode 25: reward: -58.000, steps: 60
Episo

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200


# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
